In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tqdm import tqdm_notebook

from fastmri_recon.data.fastmri_sequences import Masked2DSequence
from fastmri_recon.models.pdnet import pdnet

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [4]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# generators
AF = 4
train_gen = Masked2DSequence(train_path, af=AF, inner_slices=8, rand=True, scale_factor=1e6)
val_gen = Masked2DSequence(val_path, af=AF, scale_factor=1e6)

In [6]:
run_params = {
    'n_primal': 5,
    'n_dual': 5,
    'n_iter': 10,
    'n_filters': 32,
}

n_epochs = 10
run_id = f'pdnet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

pdnet_af4_1575124115


In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=100, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
    profile_batch=0,
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

W1130 15:28:35.583300 140408834021120 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [8]:
model = pdnet(lr=1e-3, **run_params)
print(model.summary(line_length=150))

Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
kspace_input (InputLayer)                        [(None, 640, None, 1)]           0                                                                   
______________________________________________________________________________________________________________________________________________________
buffer_primal (Lambda)                           (None, 640, None, 5)             0                 kspace_input[0][0]                                
______________________________________________________________________________________________________________________________________________________
mask_input (InputLayer)                          [(None, 640, None)]           

In [9]:
%debug

E1130 15:28:41.970839 140408834021120 interactiveshell.py:1178] No traceback has been produced, nothing to debug.


In [ ]:
model.fit_generator(
    train_gen, 
    steps_per_epoch=5, 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=1,
    verbose=0,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback,],
    max_queue_size=35,
    use_multiprocessing=True,
    workers=35,
)

In [ ]:
# %%time
# # simple overfit trials
# data = train_gen[0]
# val_data = val_gen[0]
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=data[0][0].shape[0], 
#     epochs=500,
#     verbose=2, 
#     shuffle=False,
# )